<a href="https://colab.research.google.com/github/nickprock/corso_data_science/blob/devs/machine_learning_pills/02_unsupervised/05_association_rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Association Rules

<br>

![MBA](https://miro.medium.com/max/5760/1*DHfQvlMVBaJCHpYmj1kmCw.png)

<br>

[Image Credits](https://towardsdatascience.com/market-basket-analysis-978ac064d8c6)

<br>

### Definizione

Una regola associativa collega tra di loro gli attributi di un
insieme di dati.

Ha la forma: Corpo → Testa [supporto,confidenza] dove:
* Corpo, Testa sono itemset
* supporto, confidenza sono valori percentuali

Le regole associative creano "regole" del tipo **IF - THEN** ad esempio:
* il 75% dei clienti che ha comprato la mela ha comprato anche la birra
* **IF** mela **THEN** birra

Nell'individuazione delle regole entrano in gioco alcune misure. La prima è la ***frequenza empirica*** ovvero il numero di volte in cui si presenta un Item sul numero di transazioni. Altre misure verranno descritte di seguito.

**N.B. Nelle nostre immagini la mela è detta *CORPO* della regola, la birra viene detta *TESTA***


#### Supporto

Indica la frazione delle transazioni che contiene gli item di interesse. Ad esempio la mela compare nel 50% delle transazioni. La birra nel 75% delle transazioni.

<br>

![Supporto](https://annalyzin.files.wordpress.com/2016/04/association-rule-support-table.png?w=503&h=447)

<br>

[Image Credits](https://www.kaggle.com/ashishpatel26/association-rule-mining-for-lastfm-using-python)

<br>

Il primo step di un problema da risolvere con le regole associative è trovare gli item che soddisfino una soglia di confidenza minima.

#### Confidenza

La frazione delle transazioni in cui compare l'oggetto *CORPO* contiene anche l'oggetto *TESTA*. Ad esempio su quattro transazioni in cui compare la mela in 3 compare **ANCHE** la birra. Confidenza = 75%

<br>

![Confidenza](https://annalyzin.files.wordpress.com/2016/03/association-rule-confidence-eqn.png?w=527&h=77)

<br>

[Image Credits](https://www.kaggle.com/ashishpatel26/association-rule-mining-for-lastfm-using-python)

<br>

Dopo aver generato tutti gli itemset frequenti, vanno identificate le regole forti, vale a dire quelle regole che superano una soglia di confidenza fissata.

Se l’itemset è frequente, si possono quindi ricavare le regole, ma solo alcune saranno forti.
Non sempre le regole forti sono significative e potenzialmente interessanti.

#### Lift

Per valutare la significatività di una regola si usa l’indice di lift:
* Valori di Lift > 1 indicano che la regola è più efficace nel predire la probabilità che la testa sia contenuta in una generica transazione, di quanto lo sia la sua frequenza.
* Valori di Lift < 1 indicano che la regola che nega la testa, è più efficace della regola iniziale.

<br>

![Lift](https://annalyzin.files.wordpress.com/2016/03/association-rule-lift-eqn.png?w=566&h=80)

<br>

[Image Credits](https://www.kaggle.com/ashishpatel26/association-rule-mining-for-lastfm-using-python)

<br>

### Alcuni problemi che si possono riscontrare

* Bassa confidenza: troppe regole e molte di queste poco interessanti perchè dettate da casualità
* Alta confidenza: regole molto forti che portano al loro interno ridondanza.

### Obiettivo del notebook

Questo notebook è basato sul kernel kaggle [Bakery Business Model: Association Rules ](https://www.kaggle.com/bbhatt001/bakery-business-model-association-rules). L'obiettivo è quello di capire quale è il prodotto che si vende di più in questa Bakery di Edinburgo applicando l'algoritmo [Apriori](https://en.wikipedia.org/wiki/Apriori_algorithm).

### Importare le librerie

In [0]:
import numpy as np 
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

### Dataset



In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'YOUR LINK'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('BreadBasket_DMS.csv')

In [0]:
df = pd.read_csv("BreadBasket_DMS.csv")
df.head()

### Analisi del dato e Preprocessing

In [0]:
print("il dataset contiene ", df.shape[0], " righe e ", df.shape[1], " colonne")
print("\n")
print(df.info())

Dalle informazioni nel dataset sembra non esserci nessun dato mancante ma se sicontrollano i 95 prodotti (*Item*) presenti nel menù del locale si scopre un inquietante prodotto **NONE** che compare quasi 800 volte.

Il primo lavoro di preprocessing è rimuovere le righe in cui è presente il valore mancante. Lo facciamo mediante il metodo *drop* utilizzando l'indice di riga delle osservazioni con *Item* = **NONE**.

In [0]:
print(sorted(set(df["Item"])))
print("Number of Item: ", len(set(df["Item"])))
print("NONE Item freq: ", df.loc[df['Item']=='NONE','Transaction'].count())

In [0]:
df.drop(df.loc[df['Item']=='NONE',:].index, inplace=True)
print("Number of Item: ", len(set(df["Item"])))

Vediamo i primi 10 prodotti più acquistati.

In [0]:
df['Item'].value_counts().sort_values(ascending=False).head(10)

In [0]:
fig, ax=plt.subplots(figsize=(18,10))
df['Item'].value_counts().sort_values(ascending=False).head(10).plot(kind='bar')
plt.ylabel('Number of transactions')
plt.xlabel('Items')
ax.get_yaxis().get_major_formatter().set_scientific(False)
plt.title('Prodotti più venduti')

Dividiamo la giornata in fasce orarie:
* Mattina
* Pomeriggio
* Sera
* Notte

Per vedere qual è la parte della giornata con il maggior numero di transazioni.

Vista la tipologia di attività ci aspettiamo una grossa mole di lavoro la mattina e il pomeriggio, molto meno la sera, quasi niente la notte.

In [0]:
df.loc[(df['Time']<'12:00:00'),'Daytime']='Morning'
df.loc[(df['Time']>='12:00:00')&(df['Time']<'17:00:00'),'Daytime']='Afternoon'
df.loc[(df['Time']>='17:00:00')&(df['Time']<'21:00:00'),'Daytime']='Evening'
df.loc[(df['Time']>='21:00:00')&(df['Time']<'23:50:00'),'Daytime']='Night'

In [0]:
fig, ax=plt.subplots(figsize=(18,10))
sns.set_style('darkgrid')
df.groupby('Daytime')['Item'].count().sort_values().plot(kind='bar')
plt.ylabel('Number of transactions')
ax.get_yaxis().get_major_formatter().set_scientific(False)
plt.title('Transazioni per fasce orarie')

Continuiamo l'arricchimento del dataset estraendo informazioni dalla variabile temporale. E vediamo la serie di transazioni giornaliere dell'attività.

Il periodo natalizio non è stato molto fortunato.

In [0]:
df['Date_Time']=pd.to_datetime(df['Date']+' '+df['Time'])
df['Day']=df['Date_Time'].dt.day_name()
df['Month']=df['Date_Time'].dt.month
df['Month_name']=df['Date_Time'].dt.month_name()
df['Year']=df['Date_Time'].dt.year
df['Year_Month']=df['Year'].apply(str)+' '+df['Month_name'].apply(str)

df.index=df['Date_Time']
df.index.name='Date'
df.drop(['Date_Time', 'Date','Time'],axis=1,inplace=True)
df.head()

In [0]:
fig, ax=plt.subplots(figsize=(18,10))
df['Item'].resample('D').count().plot()
plt.ylabel('Number of transactions')
plt.title('Daily business during the past months')

### Preparazione del dataset

Per applicare le regole associative, in paerticolare l'algoritmo apriori bisogna trasformare il dataframe in una lista di transazioni con gli item acquistati insieme.

In [0]:
lst=[]
for item in df['Transaction'].unique():
    lst2=list(set(df[df['Transaction']==item]['Item']))
    if len(lst2)>0:
        lst.append(lst2)
print("le prime tre transazioni: ", lst[0:3])
print("\n")
print("numero di transazioni: ",len(lst))

Per prima cosa facciamo One-Hot Encoding sulle transazioni mediante *TransactionEncoder*.

In [0]:
te=TransactionEncoder()
te_data=te.fit(lst).transform(lst)
data_x=pd.DataFrame(te_data,columns=te.columns_)
print(data_x.head())

### Applicazione dell'algoritmo

Usiamo l'algoritmo a priori con un supporto minimo pari a 0.3 quindi gli item deveono comparire nel 30% delle transazioni per essere considerati.

questo algortimo ci restituisce anche altre due metriche rispetto a quelle sopra citate:
* leverage: La differenza tra il supporto e il prodotto delle coverage.
* conviction: Misura di dipendenza della *TESTA* dal *CORPO*. Più è alto più è dipendente.

In [0]:
frequent_items= apriori(data_x, use_colnames=True, min_support=0.03)
print(frequent_items.head())

rules = association_rules(frequent_items, metric="lift", min_threshold=1)
rules.antecedents = rules.antecedents.apply(lambda x: next(iter(x)))
rules.consequents = rules.consequents.apply(lambda x: next(iter(x)))
rules

Infine usiamo la libreria networkx per trovare le connessioni tra gli item più frequenti e trovare il prodotto più influente.

[**SPOILER ALERT!!!**] Siamo in una caffetteria.

In [0]:
fig, ax=plt.subplots(figsize=(18,10))
GA=nx.from_pandas_edgelist(rules,source='antecedents',target='consequents')
nx.draw(GA,with_labels=True)
plt.show()

### Link Utili

[Bakery Business Model](https://www.kaggle.com/bbhatt001/bakery-business-model-association-rules)

[Kaggle Association Rules Mining/MBA](https://www.kaggle.com/datatheque/association-rules-mining-market-basket-analysis)

[Apriori](https://en.wikipedia.org/wiki/Apriori_algorithm)

[Association Rules and Apriori Algorithm](https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html)

[KNIME social media association](https://hub.knime.com/knime/spaces/Examples/latest/50_Applications/04_LastFM_Recommendations/01_LastFM_Recommendations)

[Slides UniBO](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&ved=2ahUKEwicnoa25IDoAhUjolwKHUZTDYQQFjAAegQIARAB&url=http%3A%2F%2Fwww-db.deis.unibo.it%2Fcourses%2FSID%2Fold%2FLezioni%2F02%2520-%2520Regole%2520associative.pdf&usg=AOvVaw1fpDtMqi4b01ZbBRSZAiKm)